In [1]:
import pandas as pd
import requests
from datetime import datetime
import json
import os
from dateutil import parser
import collections
from bs4 import BeautifulSoup
import re

The following code helps to determine whether new data is inserted in api database or not, if so then we have to pull new movies data and put that in database and update tracking parameters. 

In [2]:
# last_page =  requests.get('https://yts.torrentbay.to/api/v2/list_movies.json?page=821&limit=50').json()
first_page = requests.get('https://yts.torrentbay.to/api/v2/list_movies.json?page=6&limit=50').json()

In [3]:
movie_a =first_page['data']['movies'][0]

In [ ]:
last_page['data']['movies'][0]['id'], last_page['data']['movies'][-1]['id']

In [ ]:
first_page['data']['movies'][0]['id'], first_page['data']['movies'][-1]['id']

In [ ]:
ids = [41763,41713]
ids

In [ ]:
len(last_page['data']['movies'])

In [ ]:
def insert_new_movies(ids:list):

    page_number = 1
    new_insertion = 0
    while True:
        init_page = requests.get(f'https://yts.torrentbay.to/api/v2/list_movies.json?page={page_number}&limit=50').json()
        
        current_ids = [init_page['data']['movies'][0]['id'],init_page['data']['movies'][-1]['id']]

        if current_ids > ids:
            new_insertion = new_insertion+len(init_page['data']['movies'])
            for movie in init_page['data']['movies']:
            
        
            
                
            page_number = page_number + 1

        else:
            break

In [ ]:
new_movies = check_new_movies(ids)

In [ ]:
def validate(movie:dict, key:str):
    


Below code is for extracting movie cast based on "IMDB code"

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import time

## method to get cast list of given imdb movie id

In [ ]:
def get_cast(imdb_id : str):
    
    url = f'https://www.imdb.com/title/{imdb_id}/fullcredits'
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    table = soup.find('table', attrs={'class' : 'cast_list'})
    cast = table.find_all('a')   
    
    return re.findall(r'title="(.*?)"', str(cast))

###  The following data needed to be pulled from api response and put in warehouse
#### : imdb_code -> unique movie code given by IMDB
#### : title_long -> full movie title with year
#### : year -> released year
#### : rating -> movie rating (1-10)
#### : runtime -> in minutes
#### : genres -> can be one or more
#### : summary -> summary of movie
#### : cast -> need to scrape from IMDB
#### : mpa_rating -> movie sensorship rating
#### : language -> can be one or more

In [ ]:
[movie_a['imdb_code'], movie_a['title_long'], movie_a['year'], movie_a['rating'], movie_a['runtime'], (movie_a['genres']), movie_a['mpa_rating'], movie_a['language']]

Here is the main problem, as movie has multiple genres, we need to figure out how to insert genres without repetation.
First we need to insert "imdb_code" and "movie_title" in movies table. Then we have to check that genre exists in genre table, if so then we can skip, else insert it.

In [ ]:
import pyodbc

In [ ]:
conn_str = 'Driver={SQL Server};''Server=DESKTOP-K8VV6KV;''Database=yts_warehouse;''Trusted_Connection=yes;''autocommit=True'

In [ ]:
conn = pyodbc.connect(conn_str)

In [ ]:
cursor = conn.cursor()

In [ ]:
type(cursor)

In [ ]:
cursor.execute("""
use yts_warehouse
""");

In [ ]:
cursor.execute("""
DBCC CHECKIDENT ('[genre]', RESEED, 0);
""");

In [ ]:
cursor.execute("""
DBCC CHECKIDENT ('[cast]', RESEED, 0);
""");

### inserting into movie table

In [13]:
a = [movie_a['imdb_code'], movie_a['title_long'], movie_a['year'],movie_a['rating'], movie_a['runtime'], movie_a['mpa_rating'], movie_a['language'], movie_a['date_uploaded']]

KeyError: 'date_uploaded'

## data validation check

In [5]:
cols = ['imdb_code','title_long', 'year', 'rating', 'runtime', 'mpa_rating', 'language', 'date_uploaded', 'genres', 'summary', 'a', 'b']

In [19]:
empty_cols = {}
for col in cols:
    if col not in movie_a.keys():
        empty_cols[col] = 'None'

In [37]:
a = [movie_a['imdb_code']] +  list(empty_cols.keys())

In [40]:
empty_cols['a']

'None'

In [29]:
del movie_a['date_uploaded']

In [30]:
movie_a

{'id': 43161,
 'url': 'https://yts.torrentbay.to/movies/slim-i-2020',
 'imdb_code': 'tt13085336',
 'title': 'Slim & I',
 'title_english': 'Slim & I',
 'title_long': 'Slim & I (2020)',
 'slug': 'slim-i-2020',
 'year': 2020,
 'rating': 8.2,
 'runtime': 107,
 'genres': ['Action'],
 'summary': 'For over 50 years country music legend Slim Dusty and his wife Joy McKean trail-blazed their way across Australia performing, writing and collecting songs of the bush and its people. They created a musical legacy that to this day continues to entertain and inspire, a catalogue of plain speaking yet profoundly insightful music documenting the rural Australian experience. Theirs is perhaps one of the greatest partnerships in Australian music history. SLIM and I is a feature documentary that tells the incredible story of that partnership and of the woman who lived beside, rather than behind, the legend. Australia\'s own "Queen of Country Music", Joy McKean.',
 'description_full': 'For over 50 years cou

In [31]:
movie_a.keys()

dict_keys(['id', 'url', 'imdb_code', 'title', 'title_english', 'title_long', 'slug', 'year', 'rating', 'runtime', 'genres', 'summary', 'description_full', 'synopsis', 'yt_trailer_code', 'language', 'mpa_rating', 'background_image', 'background_image_original', 'small_cover_image', 'medium_cover_image', 'large_cover_image', 'state', 'torrents', 'date_uploaded_unix'])

In [ ]:
cursor.execute("""
insert into movies (imdb_id, title, year, rating, runtime, mpa_rating, language, date_uploaded)
VALUES (?,?,?,?,?,?,?,?)
""",
a);

In [ ]:
for i in cursor.execute("select imdb_id from movies"):
    print(i[0])

In [ ]:
movie_a['imdb_code'] in [cursor.execute("select imdb_id from movies")]

In [ ]:
for i in cursor.execute("select imdb_id from movies"):
    print(i)

### inserting into genre and movie_genre table

In [ ]:
for genre in movie_a['genres']:
    cursor.execute("""
                declare @temp_genre_id int;
                if not exists (
                                select * from genre
                                where genre_title = ?
                              )
                begin
                    insert into genre values(?)
                end
        select @temp_genre_id  = genre_id from genre where genre_title = ?
        insert into movie_genre
        values(?, @temp_genre_id)
    """, genre, genre, genre, movie_a['imdb_code'])

In [ ]:
cursor.execute('select *from genre')
for i in cursor:
    print(i)

In [ ]:
cursor.execute('select *from movie_genre')
for i in cursor:
    print(i)

In [ ]:
# before that, we need to scrape cast from imdb_page using movie id
cast_list = get_cast(movie_a['imdb_code'])
cast_list

### inserting into cast and movie_cast table

In [ ]:
for actor in cast_list:
    cursor.execute("""
                declare @temp_actor_id int;
                if not exists (
                                select * from cast
                                where actor_name = ?
                              )
                begin
                        insert into cast values(?)
                end
        select @temp_actor_id  = actor_id from cast where actor_name = ?
        insert into movie_cast
        values(?, @temp_actor_id)
    """, actor, actor, actor, movie_a['imdb_code'])

In [ ]:
cursor.execute(" select *from cast")
for i in cursor:
    print(i)

In [ ]:
cursor.execute(" select *from movie_cast")
for i in cursor:
    print(i)

### inserting into genre and movie_genre table

In [ ]:
movie_a['summary']

In [ ]:
cursor.execute("""

insert into summary (imdb_id, summary)
values (?, ?)

""", movie_a['imdb_code'], movie_a['summary'])

### Method to insert data into rdbms


In [ ]:
def load_movie(movie:dict, cursor: pyodbc.Cursor):
    
    # first, we need to extract current movie's cast list from imdb_id
    cast_list = get_cast(movie['imdb_code'])
    
    # now, let's insert data into respective tables one by one 
    
    # 1-> Movie table
    cursor.execute("""
    insert into movies (imdb_id, title, year, rating, runtime, mpa_rating, language, date_uploaded)
    VALUES (?,?,?,?,?,?,?,?)
    """,
    movie['imdb_code'], movie['title_long'], movie['year'],movie['rating'], movie['runtime'], movie['mpa_rating'], movie['language'], movie['date_uploaded']              
                  );
    
    #2-> genre and movie_genre table
    for genre in movie['genres']:
        cursor.execute("""
                    declare @temp_genre_id int;
                    if not exists (
                                    select * from genre
                                    where genre_title = ?
                                  )
                    begin
                        insert into genre values(?)
                    end
            select @temp_genre_id  = genre_id from genre where genre_title = ?
            insert into movie_genre
            values(?, @temp_genre_id)
        """, genre, genre, genre, movie['imdb_code'])
        
    #3-> cast and movie_cast
    for actor in cast_list:
        cursor.execute("""
                    declare @temp_actor_id int;
                    if not exists (
                                    select * from cast
                                    where actor_name = ?
                                  )
                    begin
                            insert into cast values(?)
                    end
            select @temp_actor_id  = actor_id from cast where actor_name = ?
            insert into movie_cast
            values(?, @temp_actor_id)
        """, actor, actor, actor, movie_a['imdb_code'])
    
    cursor.execute("""

        insert into summary (imdb_id, summary)
        values (?, ?)

        """, movie['imdb_code'], movie['summary'])

In [ ]:
%%time
load_movie(movie=movie_a, cursor=cursor)

In [ ]:
cursor.execute("""
select *from summary
""")

for i in cursor:
    print(i)

In [ ]:
def GetParams(meta_path:str) -> list:
    meta_file = pd.read_json(meta_path)
    if len(meta_file['track_ids']) == 0:
        return None
    else:
        return list(meta_file['track_ids'])

In [ ]:
def update_track_ids(meta_path: str, track_id: list) -> None:
        meta_file = pd.read_json(meta_path)
        meta_file['track_ids'] = track_id
        meta_file.to_json(meta_path)

In [ ]:
update_track_ids('../meta.json', [123, 456])

In [ ]:
cast_url = "https://www.imdb.com/title/{}/fullcredits"
def get_cast(imdb_id: str):

        url = cast_url
        page = requests.get(cast_url.format(imdb_id))
        soup = BeautifulSoup(page.content, 'html.parser')
        table = soup.find('table', attrs={'class': 'cast_list'})
        cast = table.find_all('a')
        
        return re.findall(r'title="(.*?)"', str(cast))

In [ ]:
get_cast('tt1423953')

In [11]:
def validate(movie:dict, key:str):
    if key in movie.keys():
        return movie.get(key)
    else:
        return 'None'

In [12]:
movie_a.keys()

dict_keys(['id', 'url', 'imdb_code', 'title', 'title_english', 'title_long', 'slug', 'year', 'rating', 'runtime', 'genres', 'summary', 'description_full', 'synopsis', 'yt_trailer_code', 'language', 'mpa_rating', 'background_image', 'background_image_original', 'small_cover_image', 'medium_cover_image', 'large_cover_image', 'state', 'torrents', 'date_uploaded', 'date_uploaded_unix'])

In [15]:
print(validate(movie_a, 'date_uploaded'))

None


In [14]:
del movie_a['date_uploaded']